# Analysis of first run of experiments

## Setup
In order to perform any analysis, first we must provide an Influxdb database instance with all data inserted. As we used Postgresql as our database in our experiment, the data must be migrated from postgres to influx. These are the steps we took to perform the migration:

- Step 1: create a backup of data and save it to a local machine
- Step 2: import the backup into a local database
- Step 3: export the data to a CSV file
- Step 4: import the csv file into Influxdb

### Step 1
Connect to the server where the data is stored in a postgres database.
On server:
- sudo su - postgres
- pg_dump cabackend > cadata.sql

### Step 2
Copy the created file to a local machine and import the backup into a local postgres instance.
Local machine:
- sudo su - postgres
- pg_restore -C -d cabackend ~/Projects/caanalysis/cadata.sql

### Step 3
Export the data to a CSV file, using the copy command.
Local machine:
- sudo su - postgres
- psql
- \COPY timestamp, value, sensor_id, seance_id from sensors_sensorrecord) to '/tmp/cadata.csv' with (format csv, header);

### Step 4
Import the generated csv file into influx. We used the script from https://github.com/fabio-miranda/csv-to-influxdb with some modifications as our timestamps appeared in different time formats for some reason.
Local machine:
- python csv_import.py -i cadata.csv --dbname cadata --create -tf '%Y-%m-%d %H:%M:%S.%f+02' --tagcolumns sensor_id,seance_id -b 50000

# Sensor data
The data in the influx database is presented differently from that in a classic sql database. That is why seances and sensors are only represented by ids, which is not particullary helpful, so below is the table that tells us which sensor belogns to which sensor_id.

- Id 54: fsr_03
- Id 55: fsr_04
- Id 56: proc_01
- Id 58: pir_01
- Id 59: pir_02
- Id 60: accel01_x
- Id 61: accel01_y
- Id 62: accel01_z
- Id 63: gyro01_x
- Id 64: gyro01_y
- Id 65: gyro01_z
- Id 66: pir_03
- Id 67: pir_04
- Id 68: pir_05
- Id 69: pir_06
- Id 70: hall_z_01
- Id 71: hall_z_04
- Id 72: hall_z_03
- Id 73: hall_y_01
- Id 74: hall_z_02
- Id 75: hall_x_01
- Id 76: fsr_02
- Id 77: fsr_01
- Id 78: cpuusage_01
- Id 79: cpuusage_02
- Id 80: cpuusage_03
- Id 81: cpuusage_04
- Id 82: mempercentage_01
- Id 83: netpacketssent_01
- Id 84: netpacketsreceived_01

In [37]:
# Function used for data visualization

from influxdb import InfluxDBClient
from pandas import DataFrame
from datetime import datetime
import plotly.express as px

seances = [
    {'id': 102, 'user': 1, 'experiment': 1, 'try': 1, 'start': datetime(2019, 7, 10, 15, 7), 'end': datetime(2019, 7, 10, 15, 19)},
    {'id': 103, 'user': 1, 'experiment': 2, 'try': 1, 'start': datetime(2019, 7, 10, 15, 20), 'end':  datetime(2019, 7, 10, 15, 26)},
    {'id': 104, 'user': 1, 'experiment': 3, 'try': 1, 'start': datetime(2019, 7, 10, 15, 26), 'end':  datetime(2019, 7, 10, 15, 41)},
    {'id': 105, 'user': 1, 'experiment': 1, 'try': 2, 'start': datetime(2019, 7, 10, 15, 44), 'end':  datetime(2019, 7, 10, 15, 51)},
    {'id': 106, 'user': 1, 'experiment': 2, 'try': 2, 'start': datetime(2019, 7, 10, 15, 50), 'end':  datetime(2019, 7, 10, 15, 57)},
    {'id': 107, 'user': 1, 'experiment': 3, 'try': 2, 'start': datetime(2019, 7, 10, 15, 57), 'end':  datetime(2019, 7, 10, 16, 4)},
    {'id': 108, 'user': 2, 'experiment': 1, 'try': 1, 'start': datetime(2019, 7, 10, 16, 45), 'end':  datetime(2019, 7, 10, 17, 0)},
    {'id': 109, 'user': 2, 'experiment': 2, 'try': 1, 'start': datetime(2019, 7, 10, 17, 2), 'end': datetime(2019, 7, 10, 17, 12)},
    {'id': 110, 'user': 2, 'experiment': 3, 'try': 1, 'start': datetime(2019, 7, 10, 17, 12), 'end':  datetime(2019, 7, 10, 17, 19)},
    {'id': 111, 'user': 2, 'experiment': 1, 'try': 2, 'start': datetime(2019, 7, 10, 17, 19), 'end':  datetime(2019, 7, 10, 17, 24)},
    {'id': 112, 'user': 2, 'experiment': 2, 'try': 2, 'start': datetime(2019, 7, 10, 17, 26), 'end':  datetime(2019, 7, 10, 17, 30)},
    {'id': 113, 'user': 2, 'experiment': 3, 'try': 2, 'start': datetime(2019, 7, 10, 17, 32), 'end':  datetime(2019, 7, 10, 17, 39)},
    {'id': 114, 'user': 3, 'experiment': 1, 'try': 1, 'start': datetime(2019, 7, 12, 9, 48), 'end': datetime(2019, 7, 12, 9, 59)},
    {'id': 115, 'user': 3, 'experiment': 2, 'try': 1, 'start': datetime(2019, 7, 12, 10, 0), 'end': datetime(2019, 7, 12, 10, 8)},
    {'id': 116, 'user': 3, 'experiment': 3, 'try': 1, 'start': datetime(2019, 7, 12, 10, 9), 'end': datetime(2019, 7, 12, 10, 20)},
    {'id': 117, 'user': 3, 'experiment': 1, 'try': 2, 'start': datetime(2019, 7, 12, 10, 21), 'end':  datetime(2019, 7, 12, 10, 28)},
    {'id': 118, 'user': 3, 'experiment': 2, 'try': 2, 'start': datetime(2019, 7, 12, 10, 28), 'end':  datetime(2019, 7, 12, 10, 36)},
    {'id': 119, 'user': 3, 'experiment': 3, 'try': 2, 'start': datetime(2019, 7, 12, 10, 37), 'end':  datetime(2019, 7, 12, 10, 45)},
    {'id': 122, 'user': 4, 'experiment': 1, 'try': 1, 'start': datetime(2019, 7, 12, 11, 16), 'end':  datetime(2019, 7, 12, 11, 27)},
    {'id': 124, 'user': 4, 'experiment': 2, 'try': 1, 'start': datetime(2019, 7, 12, 11, 27), 'end':  datetime(2019, 7, 12, 11, 45)},
    {'id': 125, 'user': 4, 'experiment': 3, 'try': 1, 'start': datetime(2019, 7, 12, 11, 46), 'end':  datetime(2019, 7, 12, 12, 2)}
]

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'example')
client.switch_database('cadata')

def plot_seance_data(client, sensor_ids, seance_id, sample_interval, seances, binary=False, output=False):
    seance = None
    for s in seances:
        if s["id"] == seance_id:
            seance = s
            break
    
    print("Showing seance for user {}, experiment {}, try {}".format(seance["user"], seance["experiment"], seance["try"]))
    results = []

    for i in ids:
        if binary:
            query = "select value from value where sensor_id='" + str(i) +"' and seance_id='" + str(seance['id']) + "' and time > '" + seance['start'].strftime("%Y-%m-%d %H:%M:%S") + "' and time < '" + seance['end'].strftime("%Y-%m-%d %H:%M:%S")+ "';"
        else:
            query = "select mean(value) as value from value where sensor_id='" + str(i) +"' and seance_id='" + str(seance['id']) + "' and time > '" + seance['start'].strftime("%Y-%m-%d %H:%M:%S") + "' and time < '" + seance['end'].strftime("%Y-%m-%d %H:%M:%S") + "' group by time(" + sample_interval + ");"
        result = client.query(query)
        for r in result:
            for meas in r:
                meas.update({'sensor_id': i})
                results.append(meas)

    results = DataFrame(results)
    print(len(results))
    if results.empty:
        print("NO DATA RECEIVED")
        return
    if binary:
        fig = px.scatter(results, x="time", y="value", color="sensor_id")
    else:
        fig = px.line(results, x="time", y="value", color='sensor_id')
    fig.show()
    if output:
        return results

# Force sensors data
- Id 54: fsr_03
- Id 55: fsr_04
- Id 76: fsr_02
- Id 77: fsr_01

In [ ]:
ids = [54,55,76,77]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)
plot_seance_data(client, ids, 114, sample_interval, seances)

# Accelerometer and gyroscope data
- Id 60: accel01_x
- Id 61: accel01_y
- Id 62: accel01_z
- Id 63: gyro01_x
- Id 64: gyro01_y
- Id 65: gyro01_z

In [ ]:
ids = [60,61,62]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)
plot_seance_data(client, ids, 114, sample_interval, seances)

ids = [63,64,65]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)
plot_seance_data(client, ids, 103, sample_interval, seances)

# PC monitor data
- Id 56: proc_01
- Id 78: cpuusage_01
- Id 79: cpuusage_02
- Id 80: cpuusage_03
- Id 81: cpuusage_04
- Id 82: mempercentage_01
- Id 83: netpacketssent_01
- Id 84: netpacketsreceived_01

In [ ]:
ids = [78,79,80,81]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)

ids = [83, 84]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)

ids = [82]
sample_interval = "1s"
plot_seance_data(client, ids, 102, sample_interval, seances)

# Hall data
- Id 70: hall_z_01
- Id 71: hall_z_04
- Id 72: hall_z_03
- Id 73: hall_y_01
- Id 74: hall_z_02
- Id 75: hall_x_01

In [42]:
ids = [70, 71, 72, 73, 74, 75]
sample_interval = "1s"
plot_seance_data(client, ids, 104, sample_interval, seances, binary=True)
plot_seance_data(client, ids, 107, sample_interval, seances, binary=True)

Showing seance for user 1, experiment 3, try 1
16


Showing seance for user 1, experiment 3, try 2
6


# PIR data
- Id 58: pir_01
- Id 59: pir_02
- Id 66: pir_03
- Id 67: pir_04
- Id 68: pir_05
- Id 69: pir_06

In [40]:
ids = [58, 59, 66, 67, 68, 69]
sample_interval = "1s"
plot_seance_data(client, ids, 104, sample_interval, seances, binary=True)
plot_seance_data(client, ids, 110, sample_interval, seances, binary=True)

Showing seance for user 1, experiment 3, try 1
100


Showing seance for user 2, experiment 3, try 1
52
